# Analysis Summary

Our goal from this analysis will be to predict the outcome of March Madness games. To do this, we will be using a random forest classifer that takes in historical data from NCAA tourney matchups as input and outputs predictions for the winning team for each game in an input set of March Madness games. 

### Baseline Model
The baseline model we will use to compare our model's results against will be solely tied to a team's RPI. Our baseline model poses the following hypothesis:

    That in any NCAA tournament game, the team with the lower RPI will win the game.
  
Intuitively, this is a reasonable prediction. RPI (Ratings Percentage Index), ranks teams based on their wins, losses, and strength of schedule for the past season. If team A has a lower RPI than team B at the end of a season, it's generally considered that team A's performance throughout the season has been at a higher level than team B's. For this reason, if we knew nothing else about the two teams, predicting the outcome based on RPI is a good starting point. 
### Random Forest Approach
We would like to build on our baseline model and see if we can develop an approach that more accurately predicts the outcome of games. One of the most exciting parts of March Madness is the array of upsets that occur throughout the tournament. In general, upsets occur when a team with a higher RPI beats a team with a lower RPI. We would like to create a model that performs better than our baseline model by more accurately predicting the outcome of games, in particular predicting when upsets occur. For us to accomplish this, we can utilize RPI as well as some additional attributes that provide more information surrounding each team's level of performance in the past season. These factors can be used to build a random forest in order to: 
    1. Predict when an upset is going to occur in March Madness
    2. Identify which factors are correlated to predicting the outcome of a game

We will build a random forest by passing as input a data frame where each row corresponds to an NCAA tournament game. Each row will contain data regarding each team's yearly averages and totals in statistical categories, RPI, the game's outcome, and whether or not the team with the lowest RPI won. This last piece of information will be our dependent variable. The random forest will utilize features in our training data describing the winning team and losing team's performance during the season, in order to learn which factors are tied to predicting the outcome of a game. Once the random forest has been trained, tournament data where the outcome of the each game has been excluded can be used as input to the model to generate a set of predictions for each game in that year's tournament and test the accuracy of our random forest classifer.

### Data Sources Overview
The data the was used to perform this analysis came from the Google Cloud & NCAA® ML Competition 2018-Men's Kaggle competition (Insert Link). The following is a brief summary of all the datasets that were used:
- NCAATourneyCompactResults: Contains records from each NCAA tournament game from 1985-2017, including score and region.  
- Teams: Contains information for each Division 1 (D1) basketball team including an ID, name, first and most recent year playing D1 basketball. 
- MasseyOrdinals_Prelim2018: Contains data from 2003-2017 surrounding each D1 team's rank from various rankings sources throughout. 
- RegularSeasonDetailedResults: Contains similar information as the NCAATourneyCompactResults dataset, with the addition that each row will also contain the totals in a variety of statistical categories for the winning team and losing team. These are categories that are often found in a boxscore. 

A more in depth description of each of the datasets that were used and additional datasets provided by Kaggle can be found at https://www.kaggle.com/c/mens-machine-learning-competition-2018/data


### Data Cleansing & Preparation

In order to create our random forest, we need to complete the following:
    1. Settle on a set of statistics to evaluate the performance of each team. 
    2. Create a data frame that represents a matchup that occurred in an NCAA tournament and contains our set of statistics for each team in the matchup 

To accomplish the first task, the following statistics were calculated for each team:
    - Defensive Rebounds (DRB)
    - Efficient Field Goal Percentage (EFG)
    - Free Throws Attempted (FTA)
    - Free Throw Percentage (FTP)
    - Margin of Loss (MOL)
    - Margin of Victory (MOV)
    - Offensive Rebounds (ORB)
    - Possessions (POSS)
    - Turnovers (TO)
    - Turnovers Forced (TOF)
    - Defensive Efficiency (dEff)
    - Offensive Effiency (oEff)
    - Rankings Percentage Index (RPI)
    - Conference Tourney Wins (confTournWins)
    - Wins vs. a Tournament Team (winsVsTourney)
    - Number of Games Played (numGamesPlayed)


With the exception of the last four, the yearly pre-tournament averages (regular season and conference tourney play) were found. The last three are cumulative totals, and RPI was found by using the latest ranking given to a team before the tournament. 

By parsing through the 4 datasets listed above, we can calculate the desired information for each team in a March Madness tournament from 2003-2017 as seen below:

In [1]:
matchups = getMatchupData()
matchups.head()

NameError: name 'getMatchupData' is not defined

Each row in the output data frame represents a game played in an March Madness tournament. The 16 statistics that were targeted are used features as well as some additional information to help identify which teams played and when the game occurred. A more detailed look at how the Matchups data frame was created can be seen in the getMatchupData function in the Appendix. 

### Random Forest Creation

The matchups data frame can be used as input to a random forest classifier to output predictions for a subset of NCAA tournament games. When creating a random forest classifer, a few parameters should be considered:

- _Maximum Number of Features_: The maximum number of features that our algorithm uses to create a decision tree that will be a part of our random forest. A value of sqrt(n) where n is the number of features in our input dataset was chosen. This is generally considered to be a good starting point for random forest classifiers because it allows the decision trees that are created to have a strong chance of selecting a unique set of features to train a model with. The more unique our set of decision trees are, the more confident we can be in our random forest's performance on test data. 
- _Number of Trees_: The number of trees in a random forest impacts the classifier's effectiveness for similar reason. If not enough decision trees are chosen, some features may not be included in our model and the classifer's effectiveness is hindered. The more trees in a forest, the more likely we cover the full feature space. To have a strong chance of covering the full feature space, 1000 trees were selected to be in our random forest.

For this project, random forests classifiers were created for two use cases:
    1. Predicting the outcome of all NCAA tournament games from one season between 2003-2017
    2. Predicting the outcome of all NCAA tournament championship games from 2003-2017

In order to accomplish both tasks, two methods were created, getPredictionsChips and getPredictions. Both methods behave similarly, except getPredictionChips uses championship games as its test data set to output predictions and getPredictions uses a specified year as its test data set. In each method, a data set of historical NCAA tournament games is cleaned up by removing any qualitative columns or any column specifying a result from our baseline model. Once the training and test data sets are cleaned, a random forest classifier is created using the training data and predictions are made on the test data using the generated classifier. Each method returns a tuple with the following information: 
- _Output Predictions_ (outputPreds): A dataframe describing the prediction for each matchup, the actual result, and the probability for the predicted outcome to occur
- _Baseline Model Accuracy_ (baselineAcc): The accuracy from our baseline model that predicted a game's outcome based on which team had the lower RPI
- _Random Forest Classifier Accuracy_ (modelAcc): The accuracy from our random forest classifier that predicted a game's outcome on a variety of input features

### TODO: Random Forest Classifier Results vs. Baseline Model
- 15 Year comparison with baseline model
- Championship game predictions vs. baseline model

- Two sets of results to look at
1. Results of chip game analysis
2. Results of per season analysis 

For each set of reslts, we compared the classifier's prediction accuracy to the baseline model's. In all years of games analyzed,our model always outperformed the baseline model. This tells us that we can be smarter when picking games and not simply rely on RPI.

In [416]:
indPredicts = []
outCols = ["Predict", "Actual", "A Name", "A ID", "B Name", "B ID", "Prob For", "Prob Against"]
baseAccs = []
modelAccs = []

# Chip games testing
outputPreds, baselineAcc, modelAcc = getPredictionsChips()
print("Baseline Model Accuracy: {}".format(round(baselineAcc, 2)))
print("Our Model's Accuracy: {}".format(round(modelAcc, 2)))

for row in outputPreds:
    indPredicts.append(row.tolist())
predsDF = pd.DataFrame(indPredicts, columns = outCols)
# predsDF
# pd.DataFrame(indPredicts).to_csv("data/output/chipTestResults.csv", index=False, header=True)
# outputPreds


[1 1 1 1 0 1 1 1 1 1 1 1 1 1 1]
[0 1 0 0 0 0 1 1 1 1 1 0 0 1 1]
-21.0
15
Baseline Model Accuracy: 0.53
Our Model's Accuracy: 0.6


In [208]:
indPredicts = []
outCols = ["Predict", "Actual", "L Name", "L ID", "W Name", "W ID", "Prob For", "Prob Against"]
baseAccs = []
modelAccs = []
models = []
for i in range(2003, 2018):
    model = getForestDeltas(str(i))
#     model = getForest(str(i))
    model.setFeatureImportances()
    importances = model.featureImportances
    model.setPredictions()
    models.append(model)
    outputPreds = model.getModelPredictionsAndProbs()
    baselineAcc = model.baselineAcc
    modelAcc = model.modelAcc
    
    baseAccs.append(round(baselineAcc, 3))
    modelAccs.append(round(modelAcc, 3))

    for row in outputPreds:
        indPredicts.append(row.tolist())

# pd.DataFrame(indPredicts).to_csv("data/output/results_deltas_with_seeds_D.csv", index=False, header=True)
accDF = pd.DataFrame({"Season": range(2003,2018), "Baseline": baseAccs, "RF": modelAccs})
predsDF = pd.DataFrame(indPredicts, columns = outCols)

In [209]:
print(models[1].feature_importances)

      importance
MOV     0.169894
oEff    0.145467
dEff    0.144203
ORB     0.142949
DRB     0.139772
EFG     0.131925
seed    0.125791


In [210]:
accDF

,Season,Baseline,RF
0,2003,0.672,0.688
1,2004,0.734,0.672
2,2005,0.688,0.688
3,2006,0.672,0.641
4,2007,0.797,0.734
5,2008,0.766,0.734
6,2009,0.750,0.719
7,2010,0.672,0.672
8,2011,0.687,0.627
9,2012,0.687,0.731


In [60]:
for model in models:
    df = pd.DataFrame(model.confusionMatrix)
    df.rename(columns={0:'Pred Upset', 1:'Pred No Upset'}, 
                 index={0:'Actual Upset',1:'Actual No Upset'}, 
                 inplace=True)
    print(df)

                 Pred Upset  Pred No Upset
Actual Upset              3             18
Actual No Upset           1             42
                 Pred Upset  Pred No Upset
Actual Upset              2             15
Actual No Upset           3             44
                 Pred Upset  Pred No Upset
Actual Upset              3             17
Actual No Upset           3             41
                 Pred Upset  Pred No Upset
Actual Upset              2             19
Actual No Upset           2             41
                 Pred Upset  Pred No Upset
Actual Upset              3             10
Actual No Upset           5             46
                 Pred Upset  Pred No Upset
Actual Upset              3             12
Actual No Upset           4             45
                 Pred Upset  Pred No Upset
Actual Upset              1             15
Actual No Upset           4             44
                 Pred Upset  Pred No Upset
Actual Upset              3             18
Actual No U

In [2]:

def findChampionshipMatches():
    """
    Read in NCAA tourney matchups and return data frame containing additional column denoting (True/False) if that matchup was a championship game. 
    """
    matchups = getMatchupData()
    matchups = sortMatchups(matchups)
    ## group by season and with resulting groupby obj, find whether each row equals the dayNum max for each group
    ## store result as column in matchups defining whether championship played that day
    ## able to pass in functions to transform to perform calculations for each group
    matchups["chipGame"] = matchups.groupby(['season'])['dayNum'].transform(max) == matchups['dayNum']
    return matchups

def getPredictionsChips():
    """
    Outputs predictions for all championship games from 2003-2017 using a Random Forest classifier. Baseline model takes team with lower RPI as winner. 
    Returns a tuple consisting of a data frame containing the model's prediction for every matchup in our test dataset, the baseline model's accuracy, our model's accuracy
    """
    matchups = findChampionshipMatches()
    cols = list(matchups.columns)
    
    # Create training/test data sets
    train = matchups[matchups["chipGame"] == False]
    test = matchups[matchups["chipGame"] == True]
    baselineAcc = 1.0*sum(test["baseline"]) / test.shape[0]
    
    trainLabels = np.array(train["baseline"])
    trainLabels = trainLabels.astype(int)
    testLabels = np.array(test["baseline"])
    testLabels = testLabels.astype(int)
    testNames = np.column_stack((test["aname"], test["a_id"], test["bname"], test["b_id"]))
    # Drop qualitative & output columns
    train = train.drop(["b_id", "a_id", "baseline", "bname", "aname", "season", "dayNum", "chipGame"], axis = 1)
    test = test.drop(["b_id", "a_id", "baseline", "bname", "aname", "season", "dayNum", "chipGame"], axis = 1)
    feature_names = train.columns
    trainFeatures = np.array(train)
    testFeatures = np.array(test)
    maxFeatures = int(len(feature_names)**0.5)

    rf = RandomForestClassifier(n_estimators = 1000, random_state=42, oob_score=True, max_features=maxFeatures)
    rf.fit(trainFeatures, trainLabels)
    ## Draw sample classification tree
    # drawTree(rf, "sampleTree")

    predictions = rf.predict(testFeatures)
    predictProbs = rf.predict_proba(testFeatures)
    modelAcc = 1.0*(predictions.shape[0] - sum(predictions ^ testLabels)) / predictions.shape[0]
    stack = np.column_stack((predictions.T, testLabels.T, testNames[:,0], testNames[:,1], testNames[:,2], testNames[:,3], predictProbs[:,0], predictProbs[:,1]))
    outputPreds = stack[stack[:,0].argsort()]
    return outputPreds, baselineAcc, modelAcc

### Utilize historical matchup data to build RF model. 
def getForest(year, train=None, test=None):
    """
    Outputs predictions for games from test data set using a Random Forest classifier. Baseline model takes team with lower RPI as winner. 
    Returns a tuple consisting of a data frame containing the model's prediction for every matchup in our test dataset, the baseline model's accuracy, our model's accuracy
    """
    model = mod.Model()
    matchups = getMatchupData()
    matchups = mergeRankings(matchups)
    matchups = sortMatchups(matchups)
    matchups = matchups[matchups["a_seed"] != matchups["b_seed"]]

    cols = list(matchups.columns)
    train = matchups[~matchups["b_id"].str.contains(year)]
    test = matchups[matchups["b_id"].str.contains(year)]
    baselineAcc = 1.0*sum(test["baseline"]) / test.shape[0]
    
    # Create training/test data sets
    trainLabels = np.array(train["baseline"])
    trainLabels = trainLabels.astype(int)
    testLabels = np.array(test["baseline"])
    testLabels = testLabels.astype(int)
    testNames = np.column_stack((test["aname"], test["a_id"], test["bname"], test["b_id"]))
    model.trainBaseline = trainLabels
    model.testBaseline = testLabels
    model.testNames = testNames
    
    # Drop qualitative & output columns
    train = train.drop(["b_id", "a_id", "baseline", "bname", "aname", "season", "dayNum"], axis = 1)
    test = test.drop(["b_id", "a_id", "baseline", "bname", "aname", "season", "dayNum"], axis = 1)
    # Drop insignificant quantitative columns
#     train = train.drop(["aMOV", "aMOL", "aFTA", "aFTP", "anumGamesPlayed", "aRPI", "bMOV", "bMOL", "bFTA", "bFTP", "bnumGamesPlayed", "bRPI"], axis = 1)
#     test = test.drop(["aMOV", "aMOL", "aFTA", "aFTP", "anumGamesPlayed", "aRPI", "bMOV", "bMOL", "bFTA", "bFTP", "bnumGamesPlayed", "bRPI"], axis = 1)

    train = train[['aRPI', 'bRPI', 'aMOV', 'bMOV', 'aoEff', 'boEff', 'adEff', 'bdEff', 'aDRB', 'bDRB', 'aORB', 'bORB', 'aEFG', 'bEFG']]
    test = test[['aRPI', 'bRPI', 'aMOV', 'bMOV', 'aoEff', 'boEff', 'adEff', 'bdEff', 'aDRB', 'bDRB', 'aORB', 'bORB', 'aEFG', 'bEFG']]
    
    model.train = train
    model.test = test

    feature_names = train.columns
    trainFeatures = np.array(train)
    testFeatures = np.array(test)
    maxFeatures = int(len(feature_names)**0.5)

    rf = RandomForestClassifier(n_estimators = 1000, random_state=42, oob_score=True, max_features=maxFeatures)
    rf.fit(trainFeatures, trainLabels)
    model.forest = rf
    return model

### Utilize historical matchup data to build RF model. 
def getForestDeltas(year, train=None, test=None):
    """
    Outputs predictions for games from test data set using a Random Forest classifier. Baseline model takes team with lower RPI as winner. 
    Returns a tuple consisting of a data frame containing the model's prediction for every matchup in our test dataset, the baseline model's accuracy, our model's accuracy
    """
    model = mod.Model()
    matchups = getMatchupData()
    seededMatches = mergeRankings(matchups)
    sortedMatches = sortMatchups(seededMatches)
    deltas = pd.DataFrame(findAllDeltas(sortedMatches))
    
    cols = list(deltas.columns)
    trainOrig = deltas[~deltas["b_id"].str.contains(year)]
    testOrig = deltas[deltas["b_id"].str.contains(year)]
    baselineAcc = 1.0*sum(testOrig["baseline"]) / testOrig.shape[0]
    
    # Create training/test data sets
    trainLabels = np.array(trainOrig["baseline"])
    trainLabels = trainLabels.astype(int)
    testLabels = np.array(testOrig["baseline"])
    testLabels = testLabels.astype(int)
    testNames = np.column_stack((testOrig["aname"], testOrig["a_id"], testOrig["bname"], testOrig["b_id"]))
    model.trainBaseline = trainLabels
    model.testBaseline = testLabels
    model.testNames = testNames
    
    # Drop qualitative
    train = trainOrig.drop(["b_id", "a_id", "baseline", "bname", "aname", "season", "dayNum" ], axis = 1)
    test = testOrig.drop(["b_id", "a_id", "baseline", "bname", "aname", "season", "dayNum"], axis = 1)

    train = train[['seed', 'MOV', 'oEff', 'dEff', 'DRB', 'ORB', 'EFG']]
    test = test[['seed', 'MOV', 'oEff', 'dEff', 'DRB', 'ORB', 'EFG']]
    
    model.train = train
    model.test = test

    feature_names = train.columns
    trainFeatures = np.array(train)
    testFeatures = np.array(test)
    maxFeatures = int(len(feature_names)**0.5)

    rf = RandomForestClassifier(n_estimators = 1000, random_state=42, oob_score=True, max_features=maxFeatures)
    rf.fit(trainFeatures, trainLabels)
    model.forest = rf
    return model

def drawTree(rf, treeName):
    dot_data = StringIO()
    export_graphviz(rf.estimators_[0], out_file=dot_data, filled=True, rounded=True, special_characters=True, feature_names=feature_names)
    graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
    graph.write_pdf("{}.pdf".format(treeName))
    
    
def sortRowByRPI(matchup):
    if matchup["lRPI"] > matchup["wRPI"]:
        numCols = matchup.shape[0]
        newOrder = [0, 1, 4, 5, 2, 3] + list(range(22, numCols - 3)) + list(range(6,22)) + [numCols - 2, numCols - 3, numCols - 1]
        matchup = matchup[matchup.index[newOrder]]
        return list(matchup.values)
    return list(matchup.values)

def sortRowBySeed(matchup):
    if matchup["w_seed"] > matchup["l_seed"]:
        numCols = matchup.shape[0]
        newOrder = list(range(6)) + list(range(22, numCols - 3)) + list(range(6,22)) + [numCols - 2, numCols - 3, numCols - 1]
        matchup = matchup[matchup.index[newOrder]]
        return list(matchup.values)
    elif matchup["w_seed"] == matchup["l_seed"] and matchup["wRPI"] > matchup["lRPI"]:
        numCols = matchup.shape[0]
        newOrder = list(range(6)) + list(range(22, numCols - 3)) + list(range(6,22)) + [numCols - 2, numCols - 3, numCols - 1]
        matchup = matchup[matchup.index[newOrder]]
        return list(matchup.values)
    return list(matchup.values)

def executeSeedTiebreaker(matchup):
    if matchup["w_seed"] == matchup["l_seed"]:
        matchup["baseline"] = matchup["wRPI"] < matchup["lRPI"]
    return matchup

def findDeltasForMatch(matchup):
    numCols = matchup.shape[0]
    return list(matchup.iloc[0:6].values) + list(matchup.iloc[6:22].values - matchup.iloc[22:numCols - 3].values) + [matchup.iloc[numCols - 3] - matchup.iloc[numCols - 2]] + [matchup.iloc[numCols - 1]]
        

In [3]:
def sortMatchups(matchups):
#     matchups["baseline"] = matchups["wRPI"] < matchups["lRPI"]
    matchups["baseline"] = matchups["w_seed"] < matchups["l_seed"]
    matchups = matchups.apply(executeSeedTiebreaker, axis = 1, result_type = "broadcast")
    matchups["baseline"].replace(False, 0, inplace=True)
    matchups["baseline"].replace(True, 1, inplace=True)
    sortedMatchups = matchups.apply(sortRowBySeed, axis = 1, result_type = "broadcast")

    newColNames = []
    columns = sortedMatchups.columns
    for name in columns:
        if name[0] == 'l':
            newColNames.append("a" + name[1:])
        elif name[0] == "w":
            newColNames.append("b" + name[1:])
        else:
            newColNames.append(name)
    sortedMatchups.columns = newColNames
    return sortedMatchups

def findAllDeltas(matchups):
    # expand results to extract each item from list returned each iteration of apply function
    deltas = matchups.apply(findDeltasForMatch, axis = 1, result_type = "expand")
    colNames = list(matchups.columns[0:6]) + ["DRB", "EFG", "FTA", "FTP", "MOL", "MOV", "ORB", "POSS", "RPI",
                                             "TO", "TOF", "confTourneyWins", "dEff", "numGamesPlayed", "oEff",
                                             "winsVsTourney", "seed", "baseline"]
    deltas.columns = colNames
    return deltas

def mergeRankings(matchups):
    seeds = pd.read_csv("data/NCAATourneySeeds.csv")
    seeds["id"] = seeds.TeamID.astype(str).str.cat(seeds.Season.astype(str), sep='_')
    seeds["Seed"] = seeds.Seed.str.extract('(\d+)', expand=False).astype(int)
    seeds = seeds.drop(["Season", "TeamID"], axis = 1)
    temp = matchups.merge(seeds, how='left', left_on = "w_id", right_on = "id")
    temp = temp.rename(index=str, columns={"Seed": "w_seed"})
    temp = temp.merge(seeds, how='left', left_on = "l_id", right_on = "id")
    temp = temp.rename(index=str, columns = {"Seed": "l_seed"})
    merged = temp.drop(["id_x", "id_y"], axis = 1)
    return merged

In [8]:
import pandas as pd 
import numpy as np
import team, game as g
import Model as mod
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
# Used for developing visual of Random Forest if desired
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO
from IPython.display import Image
import pydotplus

def getTeamNames():
    """
    Return dictionary where key is team ID and value is team name
    """
    names = {}
    teams = pd.read_csv("Data/Teams.csv")
    for index, row in teams.iterrows():
        teamId = row["TeamID"]
        name = row["TeamName"]
        names[teamId] = name
    return names

def getSeasonStats(ncaaTourneyTeams):
    """
    Use regular season results and RPI rankings to create a 
    dictionary where key is the team's ID and the value is a 
    Team object. Team objects contain yearly avg stats for each 
    team in various categories.
    """
    teams = {}
    names = getTeamNames()
    unfiltRanks = pd.read_csv("data/MasseyOrdinals_Prelim2018.csv")
    rankings = unfiltRanks[(unfiltRanks["SystemName"] == "RPI") & (unfiltRanks["RankingDayNum"] == 133)]
    regSeasonResults = pd.read_csv("data/RegularSeasonDetailedResults.csv")
    for index, row in regSeasonResults.iterrows():
        season = row["Season"]
        dayNum = row["DayNum"]
        wTeamId = row["WTeamID"]
        lTeamId = row["LTeamID"]
        customWId = str(wTeamId) + "_" + str(season)
        customLId = str(lTeamId) + "_" + str(season)
        wRPI = None
        lRPI = None
        try:
            wRPI = rankings[(rankings["Season"] == season) & (rankings["TeamID"] == wTeamId)].iloc[0]["OrdinalRank"]
            lRPI = rankings[(rankings["Season"] == season) & (rankings["TeamID"] == lTeamId)].iloc[0]["OrdinalRank"]
        except Exception as e:
            pass
            # print str(lTeamId) + " " + str(season) + " not found"
        
        if customWId not in teams:
            teams[customWId] = team.Team(customWId)
        if customLId not in teams:
            teams[customLId] = team.Team(customLId)
        wTeam = teams[customWId]
        wTeam.RPI = wRPI
        wTeam.name = names[wTeamId]
        wTeam.updateStats(row, True)
        if customLId in ncaaTourneyTeams:
            wTeam.winsVsTourney += 1
        lTeam = teams[customLId]
        lTeam.name = names[lTeamId]
        lTeam.RPI = lRPI
        lTeam.updateStats(row, False)
        
    for t in teams:
        teams[t].setScore()
    
    return teams

def getTeamStats(teams):
    """
    Get season stats for each team in a Data Frame. Able to see yearly averages and totals for stored
    statistical categories in each team object in teams dictionary
    """
    allTeamData = []
    visited = set()
    for team in teams:
        if team not in visited:
            idAndSeason = team.split("_")
            season = idAndSeason[1]
            _id = idAndSeason[0]
            teamData = teams[team].objToDict()
            teamData['season'] = season
            allTeamData.append(teamData)
            visited.add(team)
    return pd.DataFrame(allTeamData)

def getOrdinals(teamStats):
    """
    Get DataFrame where team stats are converted to ordinal rankings, grouped by season
    Certain categories are ranked where the min of the group is the highest ranking (Ex: RPI)
    """
    toReturn = teamStats.copy()
    copy = toReturn.copy()
    copy['season'] = copy['season'].astype(int)
    copy.drop(copy.select_dtypes(['object']), inplace=True, axis=1)
    new = copy.groupby('season').rank(axis = 1, ascending = False, method = 'min')
    new.drop(columns=['numGamesPlayed'])
    copy.update(new)
    toSwap = ['RPI', 'MOL', 'TO', 'confTournWins', 'dEff', 'winsVsTourney']
    for item in toSwap:
        copy[item] = copy.groupby('season')[item].rank(ascending = False, method = 'max')
    toReturn.update(copy)
    return toReturn

def dataToDict(df):
    """
    Convert dataframe to list to dictionary where key is team id and value is row data
    """
    data = {}
    for index, row in df.iterrows():
        data[row["_id"]] = row.to_dict()
    return data

def normalizeFeatures(teamStats):
    """
    Uses team stats for all seasons and returns data frame with stats normalized by season
    """
    toReturn = teamStats.copy()
    copy = toReturn.copy()
    copy['season'] = copy['season'].astype(int)
    copy.drop(copy.select_dtypes(['object']), inplace=True, axis=1)
    normalized = normalize(copy, 'season')
#     normalized.drop(columns=['numGamesPlayed'])
    toReturn.update(normalized)
    return toReturn

def normalize(df, by):
    """
    groups df by season and normalizes each statistical category of features from 0 - 1
    """
    groups = df.groupby(by)
    mins = groups.transform(np.min)
    maxs = groups.transform(np.max)
    return (df[mins.columns] - mins) / (maxs - mins)

def populateNCAATourneyTeams():
    """
    Create an ID for each team using a combination of its id and the season the team played in. 
    Output a dictionary with an entry for each team whose key is its newly created id
    """
    ncaaTourneyTeams = {}
    ncaaTournResults = pd.read_csv("data/NCAATourneyCompactResults.csv")
    for index, row in ncaaTournResults.iterrows():
        season = row["Season"]
        dayNum = row["DayNum"]
        wTeamId = row["WTeamID"]
        lTeamId = row["LTeamID"]
        customWId = str(wTeamId) + "_" + str(season)
        customLId = str(lTeamId) + "_" + str(season)

        if customWId not in ncaaTourneyTeams:
            ncaaTourneyTeams[customWId] = 1
        if customLId not in ncaaTourneyTeams:
            ncaaTourneyTeams[customLId] = 1
    return ncaaTourneyTeams

def getMatchups(teams, inputDataModified=False):
    """
    Use NCAA Tournament results to return data frame of matchups where each row contains data for one matchup between two teams, including their yearly avg totals in statistical categories, RPI, and game result.
    """
    matchups = []
    ncaaTournResults = pd.read_csv("data/NCAATourneyCompactResults.csv")
    for index, row in ncaaTournResults.iterrows():
        season = row["Season"]
        dayNum = row["DayNum"]
        wTeamId = row["WTeamID"]
        lTeamId = row["LTeamID"]
        customWId = str(wTeamId) + "_" + str(season)
        customLId = str(lTeamId) + "_" + str(season)

        if customWId in teams and customLId in teams:
            wTeamData, lTeamData = {}, {}
            if inputDataModified:
                wTeamData = teams[customWId].copy()
                del wTeamData['season']
            else:
                wTeamData = teams[customWId].objToDict().copy()
            wTeamDataMod, lTeamDataMod = {}, {}
            for key in wTeamData.keys():
                wTeamDataMod["w" + key] = wTeamData[key]
                
            if inputDataModified:
                lTeamData = teams[customLId]
                del lTeamData['season']
            else:
                lTeamData = teams[customLId].objToDict().copy()
            for key in lTeamData.keys():
                lTeamDataMod["l" + key] = lTeamData[key]
                
            matchupData = wTeamDataMod.copy()
            matchupData.update(lTeamDataMod)
            matchupData["dayNum"] = dayNum
            matchupData["season"] = season
            matchups.append(matchupData)
    colOrder = ["dayNum", "season", "l_id", "lname", "w_id", "wname", "lscore", "wscore", "lDRB", "lEFG", "lFTA", "lFTP", "lMOL", "lMOV", "lORB", "lPOSS",
                "lRPI", "lTO", "lTOF", "lconfTournWins", "ldEff", "lnumGamesPlayed", "loEff", "lwinsVsTourney",
                "wDRB", "wEFG", "wFTA", "wFTP", "wMOL", "wMOV", "wORB", "wPOSS", "wRPI", "wTO", "wTOF", 
                "wconfTournWins", "wdEff", "wnumGamesPlayed", "woEff", "wwinsVsTourney"]
    df = pd.DataFrame.from_dict(matchups)
    df = df[colOrder]
    return df

def getMatchupData():
    """
    Returns data frame of historical matchups in NCAA tournament.
    Reads in existing CSV if available. Otherwise, produces data frame by creating Team objects, calculating yearly avg totals for each team, and joining with historical NCAA tourney matchup data
    """
    try:
        matchups = pd.read_csv("Data/output/matchups_normalized.csv")
        return matchups
    except Exception as e:
        ncaaTourneyTeams = populateNCAATourneyTeams()
        teamObjs = getSeasonStats(ncaaTourneyTeams)
        teamStats = getTeamStats(teamObjs)
        normalized = normalizeFeatures(teamStats)
#         ordinals = getOrdinals(teamStats)
        dataAsDict = dataToDict(normalized)
        matchups = getMatchups(dataAsDict, inputDataModified=True)
#         matchups = getMatchups(teamObjs)
#         matchups.to_csv("Data/output/matchups.csv", index=False)
        matchups.to_csv("Data/output/matchups_normalized.csv", index=False)
        return matchups

In [17]:
matchups = getMatchupData()
seededMatches = mergeRankings(matchups)
sortedMatches = sortMatchups(seededMatches)

In [5]:
# ncaaTourneyTeams = populateNCAATourneyTeams()
# teamObjs = getSeasonStats(ncaaTourneyTeams)

In [38]:
matchups = getMatchups(teamObjs)

In [39]:
matchups["scoreAns"] = matchups["lscore"] < matchups["wscore"]
sum(matchups["scoreAns"])/matchups.shape[0]

0.5382262996941896

In [32]:
matchups["baseline"] = matchups["lRPI"] > matchups["wRPI"]
sum(matchups["baseline"])/matchups.shape[0]

0.6941896024464832

In [37]:
for t in teamObjs:
    curTeam = teamObjs[t]
    try:
        oScore = .5*curTeam.oEff + .4*curTeam.EFG + .2*curTeam.ORB + .05*curTeam.FTA + .1*curTeam.FTP - .25*curTeam.TO
        dScore = .5*curTeam.dEff + .2*curTeam.DRB + .25+curTeam.TOF
        curTeam.score = 3*(347 - curTeam.RPI)/347 + oScore + dScore
    except:
        curTeam.score = -1

In [41]:
# seededMatches = mergeRankings(matchups)
# seededMatches.to_csv("Data/output/matchups_w_seeds.csv", index=False)

In [73]:
# normalized = normalizeFeatures(teamStats)

In [71]:
for t in teamObjs:
    curTeam = teamObjs[t]
    try:
        oScore = .5*curTeam.oEff + .4*curTeam.EFG + .2*curTeam.ORB + .05*curTeam.FTA + .1*curTeam.FTP - .25*curTeam.TO
        dScore = .5*curTeam.dEff + .2*curTeam.DRB + .25+curTeam.TOF
        curTeam.score = 3*(347 - curTeam.RPI)/347 + oScore + dScore
    except:
        curTeam.score = -1

,DRB,EFG,FTA,FTP,MOL,MOV,ORB,POSS,RPI,TO,TOF,_id,confTournWins,dEff,name,numGamesPlayed,oEff,score,season,winsVsTourney
0,0.615147,0.405152,0.509196,0.639504,0.159649,0.494263,0.718071,0.468728,0.113497,0.312872,0.302445,1104_2003,0.00,0.357401,Alabama,0.5,0.636703,0.503933,2003,0.416667
1,0.704997,0.410256,0.364850,0.677030,0.242048,0.588337,0.608130,0.401859,0.006135,0.169950,0.288288,1328_2003,0.75,0.223913,Oklahoma,0.7,0.770873,0.536563,2003,0.666667
2,0.791451,0.387632,0.631190,0.325387,0.129125,0.472645,0.756107,0.617694,0.076687,0.361682,0.411619,1272_2003,0.50,0.260283,Memphis,0.6,0.699250,0.556056,2003,0.333333
3,0.872035,0.000000,0.676078,0.568633,0.299845,0.543267,0.774560,0.703437,0.024540,0.345096,0.355701,1393_2003,0.50,0.388375,Syracuse,0.6,0.844092,0.747117,2003,0.416667
4,0.627512,0.530322,0.675238,0.939181,0.157716,0.584616,0.682577,0.505485,0.027607,0.340357,0.180180,1266_2003,0.25,0.518251,Marquette,0.5,1.000000,0.861277,2003,0.416667
5,0.595363,0.249277,0.592144,0.669783,0.355287,0.430502,0.804348,0.635145,0.211656,0.577187,0.582583,1437_2003,0.00,0.506179,Villanova,0.7,0.689636,0.729646,2003,0.166667
6,0.504063,0.446296,0.599609,0.447721,0.504221,0.331479,0.674386,0.499813,0.355828,0.670179,0.356001,1296_2003,0.50,0.601639,N Illinois,0.8,0.684424,0.653581,2003,0.000000
7,0.618238,0.412454,0.549046,0.332835,0.489103,0.309638,0.600667,0.561650,0.558282,0.443426,0.463320,1457_2003,0.00,0.441961,Winthrop,0.5,0.666186,0.579790,2003,0.166667
8,0.806801,0.381476,0.686308,0.682307,0.076677,0.584034,0.917383,0.630460,0.009202,0.326615,0.331403,1400_2003,0.25,0.386601,Texas,0.5,0.851349,0.756995,2003,0.333333
9,0.672620,0.191067,0.552654,0.678601,0.220335,0.412410,0.660229,0.615811,0.012270,0.146435,0.265265,1208_2003,0.25,0.581049,Georgia,0.4,0.869278,0.846660,2003,0.833333


In [80]:
def createTeamObjsWithOrdinals(row):
    t = team.Team(row["_id"])
    t.DRB = row["DRB"]
    t.EFG = row["EFG"]
    t.FTA = row["FTA"]
    t.FTP = row["FTP"]
    t.MOL = row["MOL"]
    t.MOV = row["MOV"]
    t.ORB = row["ORB"]
    t.POSS = row["POSS"]
    t.RPI = row["RPI"]
    t.TO = row["TO"]
    t.TOF = row["TOF"]
    t.confTournWins = row["confTournWins"]
    t.dEff = row["dEff"]
    t.numGamesPlayed = row["numGamesPlayed"]
    t.oEff = row["oEff"]
    t.score = row["score"]
    t.season = row["season"]
    t.winsVsTourney = row["winsVsTourney"]
    t.name = row["name"]
    return t

teams = {}
for index, row in normalized.iterrows():
    teams[row["_id"]] = createTeamObjsWithOrdinals(row)
    
for t in teams:
    curTeam = teams[t]
    try:
        oScore = 5*curTeam.oEff + 4*curTeam.EFG + 2*curTeam.ORB + .5*curTeam.FTA + 1*curTeam.FTP - 2.5*curTeam.TO
        dScore = 7.5*(1-curTeam.dEff) + 2*curTeam.DRB + 2.5+curTeam.TOF
        curTeam.score = 10*(1 - curTeam.RPI) + oScore + dScore
    except:
        curTeam.score = -1

In [85]:
matchups_norm = getMatchups(teams)
matchups_norm = mergeRankings(matchups_norm)
matchups_norm["scoreAns"] = matchups_norm["lscore"] < matchups_norm["wscore"]
sum(matchups_norm["scoreAns"])/matchups_norm.shape[0]

0.6197757390417941

In [87]:
matchups_norm["baseline"] = matchups_norm["l_seed"] > matchups_norm["w_seed"]
sum(matchups_norm["baseline"])/matchups_norm.shape[0]

0.6829765545361876

In [9]:
def setupStatsDF(year):
    ### read in regular season detailed results 
    allGames = []
    regSeasonResults = pd.read_csv("data/RegularSeasonDetailedResults.csv")
    games03 = regSeasonResults[regSeasonResults["Season"] == year]
    for index, row in games03.iterrows():
        season = row["Season"]
        dayNum = row["DayNum"]
        wTeamId = row["WTeamID"]
        lTeamId = row["LTeamID"]
        customWId = str(wTeamId) + "_" + str(season)
        customLId = str(lTeamId) + "_" + str(season)
        row1 = [customWId, season, dayNum, row["WScore"], row["LScore"], row[30]] + list(row[8:21])
        row2 = [customLId, season, dayNum, row["LScore"], row["WScore"], row[17]] + list(row[21:])
        allGames.append(row1)
        allGames.append(row2)
    colNames = ["teamId", "season", "dayNum", "score", "oppScore", "TOF", "FGM", "FGA", "FGM3", "FGA3", "FTM", "FTA", "OR", "DR", "Ast", 
               "TO", "Stl", "Blk", "PF"]
    gamesDF = pd.DataFrame(allGames, columns = colNames)
    gamesDF = gamesDF.drop(["dayNum"], axis = 1)
    gamesDF["season"] = gamesDF["season"].astype("object")
    gamesDF["teamId"] = gamesDF["teamId"].astype("object")
    statDevs = gamesDF.groupby(by = "teamId").std().reset_index() ## Consider using later
    gamesDF["POS"] = gamesDF["FGA"] - gamesDF["OR"] + gamesDF["TO"] + (0.4 * gamesDF["FTA"])
    sums = gamesDF.groupby(by = "teamId").sum().reset_index()
    means = gamesDF.groupby(by = "teamId").mean().reset_index()

    sums["oEff"] = sums["score"]/sums["POS"]
    sums["dEff"] = sums["oppScore"]/sums["POS"]
    sums["eFG"] = (sums["FGM"] + (0.5 + sums["FGM3"])) / sums["FGA"]
    sums["FG%"] = sums["FGM"]/sums["FGA"]
    sums["3P%"] = sums["FGM3"]/sums["FGA3"]
    sums["3PP"] = 3*sums["FGM3"]/sums["score"]
    sums["FT%"] = sums["FTM"]/sums["FTA"]

    meanSub = means[["teamId", "FTA", "OR", "DR", "TO", "TOF", "POS", "PF"]]
    sumSub = sums[["teamId", "oEff", "dEff", "eFG", "FG%", "3P%", "FT%", "3PP"]]
    stats = sumSub.merge(meanSub, how='left', left_on = "teamId", right_on = "teamId")
    stats["season"] = stats["teamId"].apply(lambda x: x[x.index("_") + 1:])
    return stats

In [29]:
ranks = getRankings()
seeds = getSeeds()
seedsAndRanks = mergeSeedsAndRanks(seeds, ranks)
names = getTeamNames()
matchups = []
teamScoreInfo = []
visited = set()
for year in range(2003, 2018):
    stats = setupStatsDF(year)
    statsSeeds = mergeSeedsRanksStats(seedsAndRanks, stats)
    normStats = normalizeFeatures(statsSeeds)
    tournRes = pd.read_csv("data/NCAATourneyCompactResults.csv")
    tournRes = tournRes[tournRes["DayNum"] > 135]
    recentRes = tournRes[tournRes["Season"] == year]

    for index, row in recentRes.iterrows():
        season = row["Season"]
        wTeamId = row["WTeamID"]
        lTeamId = row["LTeamID"]
        customWId = str(wTeamId) + "_" + str(season)
        customLId = str(lTeamId) + "_" + str(season)
        teamW = normStats[normStats["teamId"] == customWId]
        teamL = normStats[normStats["teamId"] == customLId]
        seedRankW = [teamW["RPI"].values[0], teamW["POM"].values[0], teamW["Seed"].values[0]]
        seedRankL = [teamL["RPI"].values[0], teamL["POM"].values[0], teamL["Seed"].values[0]]
        if not teamW.empty:
            wScore = findScore(teamW, teamL)
            lScore = findScore(teamL, teamW)
            if customWId not in visited:
                teamScoreInfo.append(wScore)
                visited.add(customWId)
            if customLId not in visited:
                teamScoreInfo.append(lScore)
                visited.add(customLId)
            matchups.append([season, customWId, names[wTeamId], customLId, names[lTeamId], wScore[-1], lScore[-1]] + seedRankW + seedRankL)
matchDF = pd.DataFrame(matchups, columns = ["season", "wId", "wName", "lId", "lName", "wScore", "lScore", 
                                           "wRPI", "wPOM", "wSeed", "lRPI", "lPOM", "lSeed"])
matchDF["predicted"] = matchDF["wScore"] > matchDF["lScore"]
matchDF["seedBaseline"] = matchDF["wSeed"] < matchDF["lSeed"]
matchDF["pomBaseline"] = matchDF["wPOM"] < matchDF["lPOM"]
matchDF["rpiBaseline"] = matchDF["wRPI"] < matchDF["lRPI"]
matchDF.to_csv("data/output/matchupRes_regression.csv", index = False)
                                  

In [28]:
def getRankings():
    unfiltRanks = pd.read_csv("data/MasseyOrdinals_Prelim2018.csv")
    rankings = unfiltRanks[(unfiltRanks["SystemName"] == "POM") & (unfiltRanks["RankingDayNum"] == 133) | (unfiltRanks["SystemName"] == "RPI") & (unfiltRanks["RankingDayNum"] == 133)]
    transpose = {}
    for index, row in rankings.iterrows():
        teamId = str(row["TeamID"]) + "_" + str(row["Season"])
        if teamId in transpose:
            if row["SystemName"] == "RPI":
                transpose[teamId]["RPI"] = row["OrdinalRank"]
            else:
                transpose[teamId]["POM"] = row["OrdinalRank"]
        else:
            transpose[teamId] = {}
            if row["SystemName"] == "RPI":
                transpose[teamId]["RPI"] = row["OrdinalRank"]
            else:
                transpose[teamId]["POM"] = row["OrdinalRank"]
    ranksDF = pd.DataFrame.from_dict(orient = "index", data = transpose).reset_index()
    ranksDF.columns = ["teamId", "POM", "RPI"]
    return ranksDF

def getSeeds():
    seeds = pd.read_csv("data/NCAATourneySeeds.csv")
    seeds["teamId"] = seeds.TeamID.astype(str).str.cat(seeds.Season.astype(str), sep='_')
    seeds["Seed"] = seeds.Seed.str.extract('(\d+)', expand=False).astype(int)
    seeds = seeds.drop(["Season", "TeamID"], axis = 1)
    return seeds

def mergeSeedsAndRanks(seeds, ranks):
    return seeds.merge(ranks, how = "inner", on = "teamId")

def mergeSeedsRanksStats(seedsAndRanks, stats):
    return stats.merge(seedsAndRanks, how = "inner", on = "teamId")

def normalizeFeatures(teamStats):
    """
    Uses team stats for all seasons and returns data frame with stats normalized by season
    """
    toReturn = teamStats.copy()
    copy = toReturn.copy()
    copy['season'] = copy['season'].astype(int)
    copy.drop(copy.select_dtypes(['object']), inplace=True, axis=1)
    normalized = normalize(copy, 'season')
    toReturn.update(normalized)
    return toReturn

def normalize(df, by):
    """
    groups df by season and normalizes each statistical category of features from 0 - 1
    """
    groups = df.groupby(by)
    mins = groups.transform(np.min)
    maxs = groups.transform(np.max)
    return (df[mins.columns] - mins) / (maxs - mins)

def findScore(teamA, teamB):
    offScoreData = [5*teamA["oEff"].values[0],(1 - teamA["POS"] - teamB["POS"].values[0]).values[0],2*teamA["3PP"].values[0],3*teamA["3P%"].values[0],8*teamA["FG%"].values[0],5*teamA["FTA"].values[0],3.5*teamA["FT%"].values[0],2*teamA["OR"].values[0],2.5*teamA["TO"].values[0]]
    defScoreData = [8.5*(1-teamA["dEff"]).values[0], 5*teamA["DR"].values[0], 3*teamA["TOF"].values[0], 5*teamA["PF"].values[0]]
    seedData = [12.5*(1-teamA["Seed"]).values[0], 10*(1-teamA["POM"]).values[0], 7.5*(1-teamA["RPI"]).values[0]]
    offScore = (5*teamA["oEff"]*(1 - teamA["POS"] - teamB["POS"].values[0]) + 2*teamA["3PP"] + 3*teamA["3P%"] + 8*teamA["FG%"] + 5*teamA["FTA"] + 3.5*teamA["FT%"] + 2*teamA["OR"] - 2.5*teamA["TO"])
    defScore = (8.5*(1-teamA["dEff"]) + 5*teamA["DR"] + 3*teamA["TOF"] - 5*teamA["PF"])
    seedScore = 12.5*(1-teamA["Seed"]) + 10*(1-teamA["POM"]) + 7.5*(1-teamA["RPI"])
    total = offScore.values[0] + defScore.values[0] + seedScore.values[0]
    scoreData = [teamA["teamId"].values[0]] + offScoreData + [offScore.values[0]] + defScoreData + [defScore.values[0]] + seedData + [seedScore.values[0]] + [total]
    return scoreData



In [ ]:
#     offScoreData = [5*teamA["oEff"].values[0],(1 - teamA["POS"] - teamB["POS"].values[0]).values[0],2*teamA["3PP"].values[0],3*teamA["3P%"].values[0],8*teamA["FG%"].values[0],3*teamA["FTA"].values[0],1.5*teamA["FT%"].values[0],2*teamA["OR"].values[0],2.5*teamA["TO"].values[0]]
#     defScoreData = [7.5*(1-teamA["dEff"]).values[0], 5*teamA["DR"].values[0], 3*teamA["TOF"].values[0], 3*teamA["PF"].values[0]]
#     seedData = [5*(1-teamA["Seed"]).values[0], 7.5*(1-teamA["POM"]).values[0], 5*(1-teamA["RPI"]).values[0]]
#     offScore = (5*teamA["oEff"]*(1 - teamA["POS"] - teamB["POS"].values[0]) + 2*teamA["3PP"] + 3*teamA["3P%"] + 4*teamA["FG%"] + 3*teamA["FTA"] + 1.5*teamA["FT%"] + 2*teamA["OR"] - 2.5*teamA["TO"])
#     defScore = (6.5*(1-teamA["dEff"]) + 5*teamA["DR"] + 3*teamA["TOF"] - 3*teamA["PF"])
#     seedScore = 12*(1-teamA["Seed"]) + 10*(1-teamA["POM"]) + 7.5*(1-teamA["RPI"])
#     total = offScore.values[0] + defScore.values[0] + seedScore.values[0]
#     scoreData = [teamA["teamId"].values[0]] + offScoreData + [offScore.values[0]] + defScoreData + [defScore.values[0]] + seedData + [seedScore.values[0]] + [total]
#     return scoreData

In [11]:
# no less than for seedbaseline
matchDF.groupby('season')["predicted", "seedBaseline", "pomBaseline", "rpiBaseline"].agg('sum')

,predicted,seedBaseline,pomBaseline,rpiBaseline
season,,,,
2003,46.0,42.0,43.0,44.0
2004,49.0,47.0,46.0,47.0
2005,46.0,43.0,47.0,46.0
2006,47.0,42.0,45.0,40.0
2007,51.0,50.0,49.0,46.0
2008,48.0,47.0,51.0,46.0
2009,46.0,47.0,44.0,46.0
2010,43.0,42.0,45.0,44.0
2011,40.0,43.0,43.0,39.0


In [20]:
# 12, 10, 7.5 for seeds
old = matchDF.groupby('season')["predicted", "seedBaseline", "pomBaseline", "rpiBaseline"].agg('sum')

In [26]:
old

,predicted,seedBaseline,pomBaseline,rpiBaseline
season,,,,
2003,46.0,42.0,43.0,44.0
2004,49.0,47.0,46.0,47.0
2005,46.0,43.0,47.0,46.0
2006,47.0,42.0,45.0,40.0
2007,51.0,50.0,49.0,46.0
2008,48.0,47.0,51.0,46.0
2009,47.0,47.0,44.0,46.0
2010,43.0,42.0,45.0,44.0
2011,41.0,43.0,43.0,39.0


In [30]:
# FTA, FT% from 3, 1.5 to 5, 3.5
new = matchDF.groupby('season')["predicted", "seedBaseline", "pomBaseline", "rpiBaseline"].agg('sum')

In [31]:
new 

,predicted,seedBaseline,pomBaseline,rpiBaseline
season,,,,
2003,46.0,42.0,43.0,44.0
2004,48.0,47.0,46.0,47.0
2005,46.0,43.0,47.0,46.0
2006,46.0,42.0,45.0,40.0
2007,51.0,50.0,49.0,46.0
2008,48.0,47.0,51.0,46.0
2009,47.0,47.0,44.0,46.0
2010,46.0,42.0,45.0,44.0
2011,41.0,43.0,43.0,39.0


In [23]:
cols = ["teamId", "oEff", "POS", "3PP", "3P%", "FG%", "FTA", "FT%", "OR", "TO", "offScore", "dEff", "DR", "TOF", "PF", "defScore", "Seed", "POM", "RPI", "seedScore", "TOT"]
df = pd.DataFrame(teamScoreInfo, columns = cols)
# df[["offScore", "defScore", "seedScore"]]